In [2]:
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import numpy.linalg as linalg
import cv2
import time
assert cv2.__version__ == "3.2.0"
orb = cv2.ORB_create()


In [3]:
#function for computing the fundamental matrix
def computeF(xl, xr):
    A = []
    for i in xrange(len(xl)):
        x = xl[i]
        x_prime = xr[i]
        #Put a row into the matrix, with formula fropm class.
        A.append([x[0]*x_prime[0], x[1]*x_prime[0], x_prime[0],
                  x[0]*x_prime[1], x[1]*x_prime[1], x_prime[1], x[0], x[1], 1])
    #Get actual fundamental matrix.
    A = np.array(A)
    _, _, V = linalg.svd(A)
    F = V[-1,:].reshape((3,3))
    U, s, V = linalg.svd(F)
    sigma = np.diag(s)
    sigma[2,2] = 0
    return np.matmul(U, np.matmul(sigma, V))
    
        

In [ ]:
cap = cv2.VideoCapture('motion.mp4')
i = 0
last = None
lastDes = None
lastIm = None
while cap.isOpened():
    i += 1
    
    ret, frame = cap.read()
    if i % 10 != 0:
        continue
    if ret == False:
        break
    grayImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp, des = orb.detectAndCompute(grayImg, None)
    
    if last == None:
        last = kp
        lastDes = des
        lastIm = grayImg
        continue
    
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(lastDes, des, k=2)
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
    # cv2.drawMatchesKnn expects list of lists as matches.
    #img3 = cv2.drawMatchesKnn(lastIm,last,grayImg,kp,good, None, flags=2)
    
    last = kp
    lastDes = des
    
    time.sleep(.25)
